In [ ]:
import os
from google.cloud import videointelligence
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = #path of the json config file

In [ ]:
class audio_text:  
    def __init__(self, transcript,confidence):  
        self.transcript = transcript  
        self.confidence = confidence

In [ ]:
def write_file(content,filename):
    filename = '/home/dsailabusr1/PRAKASH/'+filename+'.txt'
    with open(filename, 'w') as f:
            for item in content:
                f.write(item.transcript+"\n")
                f.write(item.confidence+"\n")

In [ ]:
"""Transcribe speech from a video stored on GCS."""

video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.Feature.SPEECH_TRANSCRIPTION]

config = videointelligence.SpeechTranscriptionConfig(
    language_code="en-US", enable_automatic_punctuation=True
)
video_context = videointelligence.VideoContext(speech_transcription_config=config)
filename = 'Bidens Two challenges Covid and Congress  Meet The Press  NBC News'
path = 'gs://videoai_intelligence/VideoAI/'+filename+'.mp4'

operation = video_client.annotate_video(
    request={
        "features": features,
        "input_uri": path,
        "video_context": video_context,
    }
)



print("\nProcessing video for speech transcription.")
transcript = []
result = operation.result(timeout=600)

# There is only one annotation_result since only
# one video is processed.
annotation_results = result.annotation_results[0]
for speech_transcription in annotation_results.speech_transcriptions:

    # The number of alternatives for each transcription is limited by
    # SpeechTranscriptionConfig.max_alternatives.
    # Each alternative is a different possible transcription
    # and has its own confidence score.
    for alternative in speech_transcription.alternatives:

        print("Transcript: {}".format(alternative.transcript))
        print("Confidence: {}\n".format(alternative.confidence))
        transcript.append(audio_text("Transcript: {}".format(alternative.transcript),
                                    "Confidence: {}\n".format(alternative.confidence)))

write_file(transcript,filename)
print('text written to file - ',filename)
#         print("Word level information:")
#         for word_info in alternative.words:
#             word = word_info.word
#             start_time = word_info.start_time
#             end_time = word_info.end_time
#             print(
#                 "\t{}s - {}s: {}".format(
#                     start_time.seconds + start_time.microseconds * 1e-6,
#                     end_time.seconds + end_time.microseconds * 1e-6,
#                     word,
#                 )
#             )

In [ ]:
#To list available files in google storage
from google.cloud import storage

client = storage.Client()
for blob in client.list_blobs('bucket-name',prefix='prefix name'):
    print(str(blob))